# Module import

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
#hiddencell
import micropip
await micropip.install("seaborn")
# from pbl_tools import *

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

fe = fm.FontEntry(fname = 'MaruBuri-Regular.otf', name = 'MaruBuri')
fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='MaruBuri')

In [3]:
import os
import random
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# 함수 및 클래스

## **Fixed Random Seed**  

seed 값에 의해 동일한 코드를 사용해도 결과가 다를 수 있기에, 동일한 결과를 위해 seed 값을 고정시킵니다

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

## Data validation

- 유효성 검증 함수 생성

In [5]:
def making_val_table(df):
    '''입력받은 데이터셋의 유효성 검증을 위한 요약 테이블'''
    # dtypes 
    df_dtypes = df.dtypes
    
    # nunique
    df_nunique = df.nunique()
    
    # null_values
    df_nan = df.isna().sum()
    
    val_table = pd.concat([df_dtypes, df_nunique, df_nan], axis=1)
    val_table.columns = ['dtype', 'nunique', 'nan']
        
    return val_table.reset_index()

## 데이터 전처리

- 데이터 전처리 함수(사고일시) 생성

In [6]:
def convert_to_datetime(df, datetime_column='사고일시'):
    """Datetime 컬럼을 변환하고 연, 월, 일, 시간을 추출합니다."""
    df[datetime_column] = pd.to_datetime(df[datetime_column])
    df['월'] = df[datetime_column].dt.month
    df['일'] = df[datetime_column].dt.day
    df['시'] = df[datetime_column].dt.hour

    df = df.drop(columns=[datetime_column])
    return df

- 데이터 전처리 함수(장소) 생성

In [7]:
def extract_location(df, column_name='시군구'):
    location_pattern = r'(\S+) (\S+) (\S+)'

    df[['도시', '구', '동']] = df[column_name].str.extract(location_pattern)
    df = df.drop(columns=[column_name, '도시'])
    
    return df

- 데이터 전처리 함수(도로형태) 생성

In [8]:
def split_road_type(df, column_name='도로형태'):
    road_pattern = r'(.+) - (.+)'
    df[['도로형태1', '도로형태2']] = df[column_name].str.extract(road_pattern)

    df = df.drop(columns=[column_name])
    
    return df

- 데이터 전처리 함수(컬럼삭제) 생성

In [9]:
def drop_df(df, columns='ID'):
    # columns가 단일 문자열인 경우 리스트로 변환
    if isinstance(columns, str):
        columns = [columns]

    df = df.drop(columns, axis=1)
    return df

**특이사항**

- `ID` 컬럼은 unique 값으로 학습에 불필요하므로 제거합니다.
- `사고일시` 컬럼이 object로 되어 있어 datetime으로 변환이 필요합니다.

- train과 test 데이터셋 간의 컬럼 수가 서로 다릅니다.
    - test 데이터의 컬럼만을 사용하여 train 데이터를 학습합니다.
- `기상상태`, `시군구` 컬럼의 고유값 수(nunique)가 서로 다릅니다.
    - 고유값이 서로 다른 경우 test 데이터의 예측 값 생성 시 에러가 발생할 수 있습니다. 
    - 두 컬럼의 고유값 수가 다르므로 제거합니다. 
    - 현재 노트는 베이스라인 모델을 위한 노트로, 이후 본 분석에서는 train 데이터의 나머지 컬럼들과 주어진 외부 데이터들을 이용하여 추가 분석을 진행할 예정입니다.
    

| 구분|train|test |
|---|---|---|
| 컬럼 수 |23개 | 8개 |
| 기상상태 nunique | 6개 | 5개 |
| 시군구 nunique | 199개 | 192개 |

# Data Loading

## **main 데이터 불러오기**  

train.csv, test.csv 파일을 로드하여 상위행을 출력해 봅시다

In [10]:
import os # os
import csv # csv

file_path = os.path.join(r'C:\dev\dacon_study\daegu_DEV\csv')

# Data Load
train_org = pd.read_csv(os.path.join(file_path, 'train.csv'))
test_org = pd.read_csv(os.path.join(file_path, 'test.csv'))
ss = pd.read_csv(os.path.join(file_path, 'sample_submission.csv'))

display(train_org.head())
print()
display(test_org.tail())

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
10958,ACCIDENT_50567,2022-12-31 18,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차
10959,ACCIDENT_50568,2022-12-31 18,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차
10960,ACCIDENT_50569,2022-12-31 20,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차
10961,ACCIDENT_50570,2022-12-31 20,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차
10962,ACCIDENT_50571,2022-12-31 21,토요일,맑음,대구광역시 동구 효목동,교차로 - 교차로부근,건조,차대차


## **side 데이터 불러오기**

In [11]:
# side data
cctv_org = pd.read_csv(os.path.join(file_path,'cctv.csv'), encoding='EUC-KR') # 대구 CCTV 정보
park_org = pd.read_csv(os.path.join(file_path, 'car_park.csv'), encoding='EUC-KR') # 대구 주차장 정보
cpzone_org = pd.read_csv(os.path.join(file_path, 'Children_protection_zone.csv'), encoding='EUC-KR') # 대구 어린이 보호구역 정보
security_light_org = pd.read_csv(os.path.join(file_path, 'security_light.csv'), encoding='EUC-KR') # 대구 보안등 정보
# cw_accident_org = pd.read_csv(os.path.join(file_path, 'countrywide_accident.csv'), encoding='utf-8') # 타 지역 사고 이력

In [12]:
train_taas = pd.read_csv(os.path.join(file_path, 'train_taas.csv'))
test_taas = pd.read_csv(os.path.join(file_path, 'test_taas.csv'))

In [13]:
# 대구 CCTV 정보
# display(cctv_org.head())

In [14]:
# 대구 주차장 정보
# display(park_org.head())

In [15]:
# 대구 어린이 보호구역 정보
# display(cpzone_org.head())

In [16]:
# 대구 보안등 정보
# display(security_light_org.head())

In [17]:
# 타 지역 사고 이력
# display(cw_accident_org.head())

## **train, test 데이터 기간 확인하기**  

학습(train) 데이터의 기간과 예측 대상이 되는 test 데이터의 기간을 살펴 봅니다

In [18]:
display(f"train : {train_org.iloc[0]['사고일시']} ~ {train_org.iloc[-1]['사고일시']}")
display(f"test : {test_org.iloc[0]['사고일시']} ~ {test_org.iloc[-1]['사고일시']}")

'train : 2019-01-01 00 ~ 2021-12-31 23'

'test : 2022-01-01 01 ~ 2022-12-31 21'

## taas 데이터 확인

In [19]:
cols = ['사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형']

test_diff = test_taas[cols]
test_diff

,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,2022년 1월 1일 01시,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람 - 횡단중
1,2022년 1월 1일 01시,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람 - 기타
2,2022년 1월 1일 04시,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차 - 기타
3,2022년 1월 1일 04시,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차 - 기타
4,2022년 1월 1일 06시,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차 - 측면충돌
...,...,...,...,...,...,...,...
11097,2022년 12월 31일 18시,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차 - 기타
11098,2022년 12월 31일 18시,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차 - 추돌
11099,2022년 12월 31일 20시,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차 - 추돌
11100,2022년 12월 31일 20시,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차 - 기타


In [20]:
# 날짜 형식 변환
test_diff['사고일시'] = pd.to_datetime(
    test_diff['사고일시']
    .str.replace(r'년|일', '-').str.replace('월', '-')
    .str.replace('시', '').str.strip()  # 공백 제거
)

test_diff['사고일시'] = test_diff['사고일시'].dt.strftime('%Y-%m-%d %H')
test_diff['사고유형'] = test_diff['사고유형'].str.split(' - ').str[0]

test_diff

,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람
1,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람
2,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차
3,2022-01-01 04,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차
4,2022-01-01 06,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차
...,...,...,...,...,...,...,...
11097,2022-12-31 18,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차
11098,2022-12-31 18,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차
11099,2022-12-31 20,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차
11100,2022-12-31 20,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차


In [21]:
test_diff2 = test_org[cols]
test_diff2

,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람
1,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람
2,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차
3,2022-01-01 04,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차
4,2022-01-01 06,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차
...,...,...,...,...,...,...,...
10958,2022-12-31 18,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차
10959,2022-12-31 18,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차
10960,2022-12-31 20,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차
10961,2022-12-31 20,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차


In [26]:
# a_df에만 있는 행 찾기
diff_in_a = pd.merge(test_diff, test_diff2, how='outer', indicator=True).loc[lambda x: x['_merge'] == 'left_only']

# 결과 출력
diff_in_a

,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,_merge
0,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람,both
1,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람,both
2,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차,both
3,2022-01-01 04,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차,both
4,2022-01-01 06,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차,both
...,...,...,...,...,...,...,...,...
10994,2022-12-31 18,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차,both
10995,2022-12-31 18,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차,both
10996,2022-12-31 20,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차,both
10997,2022-12-31 20,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차,both


In [ ]:
# test_taas에 새 열 추가
test_taas['Is_In_Diff_in_A'] = False

# diff_in_a의 인덱스에 해당하는 행을 True로 설정
test_taas.loc[diff_in_a.index, 'Is_In_Diff_in_A'] = True

test_taas

In [ ]:
# 데이터프레임을 CSV 파일로 저장
test_taas.to_csv('test_taas_with_diff_indicator.csv', index=False)

## 데이터 유효성 검사

In [ ]:
# 기상상태, 시군구 데이터의 unique 값이 각각 7vs6, 199vs 192로 다르다
train_info = making_val_table(train_org)
test_info = making_val_table(test_org)

pd.merge(left=train_info, right=test_info, on='index', 
         how='left', suffixes=('_train','_test')).set_index('index')

## **예측 대상 확인하기**

sample_submission.csv를 로드하여 출력 형식을 확인해 봄으로서,
대회 페이지 규칙에 기술되어 있는, 예측 대상이 무엇인지 확인해 봅시다.

최종적으로 예측해야 하는 값은 'ECLO'(인명피해 심각도) 값입니다.

그러나, train데이터 컬럼에 있는 '사망자수', '중상자수', '경상자수', '부상자수'를 각각 예측하여 ECLO 예측값을 구할 수도 있습니다. 즉, ECLO 값을 예측하는 단일 모델로 접근할 수도,  각각을 별도로 예측하는 모델로 접근할 수도 있습니다.

<img src="https://drive.google.com/uc?id=19PvSqu8k04D3a-Bskcl-d-Xkckkn2Fvv">  

In [ ]:
sample_submission = pd.read_csv(os.path.join(file_path,"sample_submission.csv"))
sample_submission.head()

# Data preprocessing

- Dataframe

In [ ]:
# target 변수 지정
y_train = train_org['ECLO'].copy()

# train, test 데이터간 컬럼 동기화 
cols = test_org.columns
X_train = train_org[cols].copy()
X_test = test_org.copy()

## ID 컬럼 삭제

In [ ]:
X_test = X_test.drop(['ID'], axis=1)
X_train = X_train.drop(['ID'], axis=1)

## **파생 변수 생성 1 : 날짜, 시간정보 생성**

'사고일시' 컬럼으로 부터 연도, 월, 일, 시간 정보 추출 및 변환.

In [ ]:
X_train = convert_to_datetime(X_train)
X_test = convert_to_datetime(X_test)

## **파생 변수 생성 2 : 공간(위치) 정보 생성**

'시군구' 컬럼으로부터 의미 있는 공간 정보를 추출 및 변환 합니다

In [ ]:
X_train = extract_location(X_train)
X_test = extract_location(X_test)

## **파생 변수 추출 3 : 도로 형태 정보 추출**  

'도로형태' 컬럼은 '단일로 - 기타'와 같은 패턴으로 구성되어 있습니다. 이를 두종류의 독립된 정보로 보고 두개의 컬럼으로 분리하여 생성합니다.

In [ ]:
X_train = split_road_type(X_train)
X_test = split_road_type(X_test)

## **전처리 결과 확인**

지금까지 전처리한 결과를 확인해 봅시다

- 전처리 후 shape 확인

In [ ]:
print(f'X_train 데이터 shape : {X_train.shape}')
print(f'y_train 데이터 shape : {y_train.shape}')
print()
print(f'X_test 데이터 shape : {X_test.shape}')

In [ ]:
display(X_train.head())
display(X_test.tail())

In [ ]:
from sklearn.preprocessing import OneHotEncoder
categorical_columns = ['요일', '기상상태', '노면상태', '사고유형', '구', '동', '도로형태1', '도로형태2']

# OneHotEncoder 초기화
encoder = OneHotEncoder()

In [ ]:
# 학습과 테스트 데이터셋에서 범주형 컬럼의 고유값을 합칩니다.
unique_values = pd.concat([X_train[categorical_columns], X_test[categorical_columns]], ignore_index=True).drop_duplicates()

# 고유값들에 대해 원-핫 인코딩을 학습합니다.
encoder.fit(unique_values)

In [ ]:
# 학습 데이터셋과 테스트 데이터셋에 원-핫 인코딩을 적용합니다.
train_encoded = encoder.transform(X_train[categorical_columns])
test_encoded = encoder.transform(X_test[categorical_columns])

In [ ]:
# 원-핫 인코딩된 데이터를 밀집 행렬 형태로 변환합니다.
train_encoded_dense = train_encoded.toarray()
test_encoded_dense = test_encoded.toarray()

In [ ]:
# 원-핫 인코딩된 데이터를 DataFrame으로 변환합니다.
train_encoded_df = pd.DataFrame(train_encoded_dense, columns=encoder.get_feature_names_out())
test_encoded_df = pd.DataFrame(test_encoded_dense, columns=encoder.get_feature_names_out())

In [ ]:
# 원-핫 인코딩 전 컬럼을 제거합니다.
X_train = X_train.drop(categorical_columns, axis=1)
X_test = X_test.drop(categorical_columns, axis=1)

In [ ]:
# 원-핫 인코딩된 데이터를 원래 데이터셋에 결합합니다.
X_train = X_train.join(train_encoded_df)
X_test = X_test.join(train_encoded_df)

In [ ]:
print(f'X_train 데이터 shape : {X_train.shape}')
print(f'y_train 데이터 shape : {y_train.shape}')
print()
print(f'X_test 데이터 shape : {X_test.shape}')

# Model Train

## 모델링

- 데이터 분리
  

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train , test_size=0.2, random_state=42)

- 모델 학습

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

y_train_pred = rf.predict(X_train)
y_val_pred = rf.predict(X_val)

## 평가

In [ ]:
from sklearn.metrics import mean_squared_log_error

rmsle_train = np.sqrt(mean_squared_log_error(y_train, y_train_pred))
rmsle_val = np.sqrt(mean_squared_log_error(y_val, y_val_pred))

In [ ]:
print('train rmsle :', rmsle_train)
print('val rmsle :', rmsle_val)

## 제출

In [ ]:
y_pred = rf.predict(X_test)
y_pred

- 제출 파일형식으로 변환 후 제출 파일 생성

In [ ]:
ss['ECLO'] = y_pred
ss.head()

In [ ]:
OUTPUT_PATH = './output/'
ss.to_csv(f'{OUTPUT_PATH}submission.csv', index=False)

## **Autogluon 모델 및 제출하기**

In [ ]:
save_path = 'agModels-predictClass'

predictor = TabularPredictor(label='ECLO', 
                             eval_metric="rmse", 
                             path=save_path).fit(train_data=train_df, 
                                                 verbosity=2, 
                                                 presets='best_quality', 
                                                 auto_stack=True,
                                                 excluded_model_types=['CatBoost'])


In [ ]:
ld_board = predictor.leaderboard(silent=True)
print(ld_board)


In [ ]:
ld_board = predictor.leaderboard(train_df, silent=True)
print(ld_board)

In [ ]:
predictor = TabularPredictor.load(save_path)
pred_y = predictor.predict(X_test)

In [ ]:
pred_y

In [ ]:
# 제출 파일 생성
baseline_submission = sample_submission.copy()
baseline_submission['ECLO'] = pred_y
baseline_submission

In [ ]:
# 답안지 저장 및 제출하기
baseline_submission.to_csv(os.path.join(file_path, './baseline_submit.csv'), index=False)